In [1]:
import subprocess
import sys

In [2]:
def install_packages():
    packages = [
        "fastapi",
        "uvicorn",
        "gradio",
        "transformers",
        "torch",
        "PyPDF2",
        "python-multipart",
        "pydantic",
        "accelerate",
        "nest-asyncio"
    ]

    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Uncomment the line below when running in Colab for the first time
install_packages()

In [3]:
import os
import re
import PyPDF2
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from fastapi import FastAPI, File, UploadFile, HTTPException
from pydantic import BaseModel
from typing import List, Dict, Optional
import nest_asyncio
import uvicorn
from io import BytesIO
import threading
import time

In [4]:
# Enable nested asyncio for Colab compatibility
nest_asyncio.apply()

class SmartSDLC:
    def __init__(self):
        self.model_name = "ibm-granite/granite-3.3-2b-instruct"
        self.tokenizer = None
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.load_model()

    def load_model(self):
        """Load the Granite model for text generation"""
        print("🧠 Loading Granite 3.3 2B Instruct model...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None
            )
            print(f"✅ Model loaded successfully on {self.device}")
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            raise

    def generate_response(self, prompt: str, max_length: int = 512) -> str:
        """Generate response using the Granite model"""
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
            if torch.cuda.is_available():
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=inputs['input_ids'].shape[1] + max_length,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    num_return_sequences=1
                )

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            # Remove the input prompt from the response
            response = response[len(prompt):].strip()
            return response

        except Exception as e:
            return f"Error generating response: {str(e)}"

    def extract_pdf_text(self, file_content: bytes) -> str:
        """Extract text from PDF file"""
        try:
            pdf_reader = PyPDF2.PdfReader(BytesIO(file_content))
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
            return text.strip()
        except Exception as e:
            return f"Error extracting PDF text: {str(e)}"

    def clean_extracted_text(self, text: str) -> str:
        """Clean and format extracted PDF text"""
        import re

        # Remove extra spaces and join words properly
        cleaned = re.sub(r'\s+', ' ', text)

        # Fix sentence breaks
        cleaned = re.sub(r'([.!?])\s*([A-Z])', r'\1 \2', cleaned)

        # Fix common PDF extraction issues
        cleaned = re.sub(r'([a-z])([A-Z])', r'\1 \2', cleaned)  # Add space between camelCase
        cleaned = re.sub(r'(\w)(\d)', r'\1 \2', cleaned)  # Add space between word and number
        cleaned = re.sub(r'(\d)([A-Z])', r'\1 \2', cleaned)  # Add space between number and word

        # Clean up multiple spaces again
        cleaned = re.sub(r'\s+', ' ', cleaned)

        return cleaned.strip()


    def parse_classification_response(self, response: str) -> Dict[str, List[str]]:
        """Parse the classification response into structured format - FIXED VERSION"""
        stages = {
            "Planning": [],
            "Design": [],
            "Implementation": [],
            "Testing": [],
            "Maintenance": []
        }

        current_stage = None
        lines = response.split('\n')
        current_requirement = ""  # Buffer for multi-line requirements

        for line in lines:
            line = line.strip()

            # Skip empty lines
            if not line:
                # If we have a buffered requirement, add it
                if current_requirement and current_stage:
                    clean_req = current_requirement.strip()
                    if len(clean_req) > 10:  # Only add substantial requirements
                        stages[current_stage].append(clean_req)
                    current_requirement = ""
                continue

            # Check for stage headers
            stage_found = None
            for stage in stages.keys():
                patterns = [
                    f"{stage.upper()}:",
                    f"**{stage.upper()}:**",
                    f"**{stage}:**",
                    f"## {stage}",
                    f"### {stage}",
                    f"{stage}:",
                    stage.upper()
                ]

                for pattern in patterns:
                    if line.upper().startswith(pattern.upper()) or line.upper() == pattern.upper():
                        stage_found = stage
                        break
                if stage_found:
                    break

            if stage_found:
                # Save any previous requirement before switching stages
                if current_requirement and current_stage:
                    clean_req = current_requirement.strip()
                    if len(clean_req) > 10:
                        stages[current_stage].append(clean_req)

                current_stage = stage_found
                current_requirement = ""
                continue

            # Process requirement lines
            if current_stage and line:
                # Check if it's a bullet point or numbered item
                if (line.startswith(('- ', '* ', '• ', '→ ')) or
                    re.match(r'^\d+\.\s+', line)):

                    # Save previous requirement if exists
                    if current_requirement:
                        clean_req = current_requirement.strip()
                        if len(clean_req) > 10:
                            stages[current_stage].append(clean_req)

                    # Start new requirement, removing bullet/number
                    current_requirement = re.sub(r'^[-*•→\d\.\s]+', '', line).strip()

                else:
                    # This might be a continuation of the previous requirement
                    # or a standalone requirement without bullets
                    if current_requirement:
                        # Add to existing requirement (multi-line)
                        current_requirement += " " + line
                    else:
                        # Start new requirement without bullet
                        current_requirement = line

        # Don't forget the last requirement
        if current_requirement and current_stage:
            clean_req = current_requirement.strip()
            if len(clean_req) > 10:
                stages[current_stage].append(clean_req)

        # Clean up and validate requirements
        for stage in stages:
            # Remove duplicates and filter out single words or very short phrases
            cleaned_reqs = []
            for req in stages[stage]:
                req = req.strip()
                # Skip if it's just a single word, too short, or looks like a stage name
                if (len(req) > 15 and
                    len(req.split()) > 2 and  # At least 3 words
                    not req.upper() in [s.upper() for s in stages.keys()] and
                    req not in cleaned_reqs):
                    cleaned_reqs.append(req)

            stages[stage] = cleaned_reqs

        return stages

    def extract_sentence_based_requirements(self, text: str, existing_classification: Dict[str, List[str]]) -> Dict[str, List[str]]:
        """Extract requirements by analyzing complete sentences from the original text"""

        # Keywords for each stage
        stage_keywords = {
            "Planning": ["timeframe", "timeline", "deadline", "stakeholder", "budget", "scope", "project", "month", "scalable", "cost-effective", "expected", "goal"],
            "Design": ["interface", "design", "architecture", "database", "UI", "UX", "responsive", "MVC", "normalized", "entities", "user-friendly", "role"],
            "Implementation": ["backend", "frontend", "Python", "FastAPI", "PostgreSQL", "JWT", "authentication", "framework", "technology", "API", "developed", "implemented"],
            "Testing": ["testing", "test", "pytest", "integration", "unit", "load", "JMeter", "usability", "performance", "quality", "simulate", "concurrent"],
            "Maintenance": ["maintenance", "update", "backup", "monitor", "support", "logs", "deployment", "feedback", "review", "Sentry", "automated", "monthly"]
        }

        # Split text into sentences using multiple delimiters
        sentences = re.split(r'[.!?]+', text)

        for sentence in sentences:
            sentence = sentence.strip()

            # Skip very short sentences
            if len(sentence) < 30 or len(sentence.split()) < 5:
                continue

            # Find the best matching stage based on keyword density
            best_stage = None
            max_score = 0

            for stage, keywords in stage_keywords.items():
                score = sum(1 for keyword in keywords if keyword.lower() in sentence.lower())
                # Normalize score by sentence length to avoid bias toward long sentences
                normalized_score = score / max(len(sentence.split()) / 10, 1)

                if normalized_score > max_score and score >= 2:  # At least 2 keyword matches
                    max_score = normalized_score
                    best_stage = stage

            # Add to classification if we found a good match and don't have too many already
            if best_stage and len(existing_classification[best_stage]) < 8:
                # Clean the sentence
                clean_sentence = sentence.strip()

                # Check for duplicates (compare first 30 characters)
                is_duplicate = any(
                    clean_sentence[:30].lower() in existing_req[:30].lower() or
                    existing_req[:30].lower() in clean_sentence[:30].lower()
                    for existing_req in existing_classification[best_stage]
                )

                if not is_duplicate and len(clean_sentence) > 20:
                    existing_classification[best_stage].append(clean_sentence)

        return existing_classification

    def enhanced_classify_requirements(self, requirements_text: str) -> Dict[str, List[str]]:
        """Enhanced classification with better sentence-level parsing"""

        if len(requirements_text) > 3000:
            requirements_text = requirements_text[:3000] + "..."

        # Use a more structured prompt that encourages complete sentences
        prompt = f"""
You are a software requirements analyst. Extract complete, meaningful requirements from this document.

Document:
{requirements_text}

Classify each COMPLETE requirement into SDLC stages. Each requirement should be a full sentence or phrase that makes sense on its own.

IMPORTANT FORMATTING RULES:
- Write each requirement as a complete, meaningful statement
- Each requirement should be 10+ words long
- Don't break requirements into single words
- Use bullet points (- ) for each requirement

Example format:
PLANNING:
- The project must be completed within a three-month timeframe with adequate resource allocation
- The system should handle over 5,000 students across multiple departments and courses

DESIGN:
- The application should provide a clean and user-friendly interface with three distinct user roles
- The backend database must use normalized relational tables for students, courses, and attendance

IMPLEMENTATION:
- The backend should be developed using Python FastAPI framework with PostgreSQL database
- JWT-based authentication must be implemented to ensure secure user sessions

TESTING:
- Unit testing should be performed on individual backend functions using pytest framework
- Load testing must simulate concurrent access by at least 100 users using JMeter

MAINTENANCE:
- Monthly updates should be scheduled to accommodate curriculum changes and new courses
- Daily automated backups of the PostgreSQL database should be stored securely in the cloud

Now analyze the document and provide complete requirements:
"""

        response = self.generate_response(prompt, max_length=1200)
        print(f"🔍 Full AI Response for debugging:\n{response}\n" + "="*50)

        classification = self.parse_classification_response(response)

        # Additional validation and sentence-based extraction
        classification = self.extract_sentence_based_requirements(requirements_text, classification)

        return classification

    def generate_code(self, requirements: str, language: str = "Python") -> str:
        """Generate code based on implementation requirements"""
        prompt = f"""
Generate {language} code based on the following implementation requirements:

{requirements}

Please provide clean, well-commented, and functional code that addresses these requirements.
Include proper error handling and follow best practices.

Code:
"""

        response = self.generate_response(prompt, max_length=1000)
        return response

    def fix_bugs(self, code: str) -> str:
        """Identify and suggest fixes for bugs in code"""
        prompt = f"""
Analyze the following code for bugs, syntax errors, and logic issues.
Provide the corrected code with explanations of what was fixed.

Original Code:
{code}

Please provide:
1. List of identified issues
2. Corrected code
3. Explanation of fixes

Response:
"""

        response = self.generate_response(prompt, max_length=1000)
        return response

    def summarize_code(self, code: str) -> str:
        """Generate a human-readable summary of code"""
        prompt = f"""
Provide a clear, human-readable summary and explanation of the following code.
Explain what it does, how it works, and any important details that would help someone understand it.

Code:
{code}

Summary and Explanation:
"""

        response = self.generate_response(prompt, max_length=600)
        return response

    def generate_stage_summary(self, stage: str, requirements: List[str]) -> str:
        """Generate summary for a specific SDLC stage"""
        requirements_text = "\n".join([f"- {req}" for req in requirements])

        prompt = f"""
Generate a concise summary for the {stage} stage of the Software Development Lifecycle.
Base the summary on these requirements:

{requirements_text}

Provide a clear, professional summary that covers the key aspects and deliverables for this stage.

{stage} Stage Summary:
"""

        response = self.generate_response(prompt, max_length=400)
        return response

In [5]:
# Initialize the SmartSDLC application
print("🚀 Initializing SmartSDLC...")
smart_sdlc = SmartSDLC()

# FastAPI Backend
app = FastAPI(title="SmartSDLC API", version="1.0.0")

class CodeRequest(BaseModel):
    code: str
    language: Optional[str] = "Python"

class RequirementsRequest(BaseModel):
    requirements: str
    language: Optional[str] = "Python"

@app.post("/classify-requirements")
async def classify_requirements_endpoint(file: UploadFile = File(...)):
    """Classify requirements from uploaded PDF"""
    try:
        content = await file.read()
        text = smart_sdlc.extract_pdf_text(content)
        classification = smart_sdlc.enhanced_classify_requirements(text)
        return {"success": True, "classification": classification, "extracted_text": text}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/generate-code")
async def generate_code_endpoint(request: RequirementsRequest):
    """Generate code from requirements"""
    try:
        code = smart_sdlc.generate_code(request.requirements, request.language)
        return {"success": True, "generated_code": code}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/fix-bugs")
async def fix_bugs_endpoint(request: CodeRequest):
    """Fix bugs in provided code"""
    try:
        fixed_code = smart_sdlc.fix_bugs(request.code)
        return {"success": True, "fixed_code": fixed_code}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/summarize-code")
async def summarize_code_endpoint(request: CodeRequest):
    """Summarize provided code"""
    try:
        summary = smart_sdlc.summarize_code(request.code)
        return {"success": True, "summary": summary}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Gradio Interface Functions
def classify_requirements_interface(file):
    """Gradio interface for requirement classification"""
    if file is None:
        return "Please upload a PDF file.", {}

    try:
        with open(file.name, 'rb') as f:
            content = f.read()

        # Extract text from PDF
        text = smart_sdlc.extract_pdf_text(content)

        if not text or len(text.strip()) < 10:
            return "Could not extract meaningful text from PDF. Please check if the PDF contains readable text.", {}

        print(f"📄 Extracted text length: {len(text)}")

        # Use the enhanced classification method
        classification = smart_sdlc.enhanced_classify_requirements(text)

        print(f"📊 Classification result: {classification}")

        # Format output for display
        output = "## 📋 Requirements Classification Results\n\n"
        output += f"**📄 Extracted Text Preview:** {text[:200]}...\n\n"

        total_requirements = sum(len(reqs) for reqs in classification.values())

        if total_requirements == 0:
            output += "⚠️ **No requirements were automatically classified.**\n\n"
            # Show first few sentences for manual review
            sentences = [s.strip() for s in text.split('.') if len(s.strip()) > 30][:5]
            output += "**📝 Key sentences from document:**\n"
            for i, sentence in enumerate(sentences, 1):
                output += f"{i}. {sentence}.\n"
        else:
            output += f"**📊 Total Requirements Found: {total_requirements}**\n\n"
            for stage, requirements in classification.items():
                emoji_map = {
                    "Planning": "📋",
                    "Design": "🎨",
                    "Implementation": "💻",
                    "Testing": "🧪",
                    "Maintenance": "🔧"
                }
                output += f"### {emoji_map.get(stage, '📌')} {stage} ({len(requirements)} items)\n"
                if requirements:
                    for req in requirements:
                        output += f"- {req}\n"
                else:
                    output += "- No requirements identified for this stage\n"
                output += "\n"

        return output, classification

    except Exception as e:
        error_msg = f"❌ Error processing file: {str(e)}"
        print(f"Error in classify_requirements_interface: {e}")
        return error_msg, {}

def generate_code_interface(requirements, language):
    """Gradio interface for code generation"""
    if not requirements.strip():
        return "Please provide implementation requirements."

    try:
        code = smart_sdlc.generate_code(requirements, language)
        return f"```{language.lower()}\n{code}\n```"
    except Exception as e:
        return f"Error generating code: {str(e)}"

def fix_bugs_interface(code):
    """Gradio interface for bug fixing"""
    if not code.strip():
        return "Please provide code to analyze for bugs."

    try:
        fixed_code = smart_sdlc.fix_bugs(code)
        return fixed_code
    except Exception as e:
        return f"Error fixing bugs: {str(e)}"

def summarize_code_interface(code):
    """Gradio interface for code summarization"""
    if not code.strip():
        return "Please provide code to summarize."

    try:
        summary = smart_sdlc.summarize_code(code)
        return summary
    except Exception as e:
        return f"Error summarizing code: {str(e)}"

def generate_stage_summary_interface(stage, requirements_text):
    """Gradio interface for stage summary generation"""
    if not requirements_text.strip():
        return "Please provide requirements for the selected stage."

    try:
        requirements_list = [req.strip() for req in requirements_text.split('\n') if req.strip()]
        summary = smart_sdlc.generate_stage_summary(stage, requirements_list)
        return summary
    except Exception as e:
        return f"Error generating summary: {str(e)}"

# Create Gradio Interface
def create_gradio_interface():
    """Create the main Gradio interface"""

    with gr.Blocks(title="SmartSDLC - AI-Powered SDLC Automation", theme=gr.themes.Soft()) as interface:

        gr.Markdown("""
        # 🚀 SmartSDLC - AI-Powered Software Development Lifecycle Automation

        Automate key stages of your software development process with AI-powered tools.
        **Powered by IBM Granite 3.3 2B Instruct model** 🧠

        **✨ Enhanced Features:**
        - 🔍 **Improved Requirements Classification** - Complete sentence extraction
        - 📋 **Multi-pass Analysis** - Better accuracy with fallback methods
        - 🎯 **Stage-specific Targeting** - Focused extraction for each SDLC phase
        """)

        with gr.Tabs():

            # Requirements Classifier Tab
            with gr.TabItem("📋 Requirements Classifier"):
                gr.Markdown("""
                ### Upload a PDF containing software requirements to classify them into SDLC stages.

                **📝 Supported formats:** PDF files with readable text
                **🎯 Output:** Complete requirements organized by Planning, Design, Implementation, Testing, and Maintenance stages
                """)

                with gr.Row():
                    with gr.Column():
                        pdf_input = gr.File(
                            label="📄 Upload Requirements PDF",
                            file_types=[".pdf"],
                            type="filepath"
                        )
                        classify_btn = gr.Button("🔍 Classify Requirements", variant="primary", size="lg")

                    with gr.Column():
                        classification_output = gr.Markdown(label="📊 Classification Results")

                classify_btn.click(
                    fn=classify_requirements_interface,
                    inputs=[pdf_input],
                    outputs=[classification_output, gr.State()]
                )

            # AI Code Generator Tab
            with gr.TabItem("💻 AI Code Generator"):
                gr.Markdown("""
                ### Generate code based on implementation requirements.

                **🎯 Input:** Natural language description of what you want to build
                **💻 Output:** Clean, commented code in your chosen language
                """)

                with gr.Row():
                    with gr.Column():
                        requirements_input = gr.Textbox(
                            label="📝 Implementation Requirements",
                            placeholder="Describe what you want the code to do...",
                            lines=5
                        )
                        language_dropdown = gr.Dropdown(
                            choices=["Python", "JavaScript", "Java", "C++", "C#"],
                            value="Python",
                            label="🔧 Programming Language"
                        )
                        generate_btn = gr.Button("🚀 Generate Code", variant="primary", size="lg")

                    with gr.Column():
                        generated_code = gr.Markdown(label="💻 Generated Code")

                generate_btn.click(
                    fn=generate_code_interface,
                    inputs=[requirements_input, language_dropdown],
                    outputs=[generated_code]
                )

            # Bug Fixer Tab
            with gr.TabItem("🐛 Bug Fixer"):
                gr.Markdown("""
                ### Identify and fix bugs in your code.

                **🔍 Analysis:** Detects syntax errors, logic issues, and common bugs
                **🔧 Output:** Corrected code with detailed explanations
                """)

                with gr.Row():
                    with gr.Column():
                        code_input = gr.Textbox(
                            label="🐛 Code with Bugs",
                            placeholder="Paste your code here...",
                            lines=10
                        )
                        fix_btn = gr.Button("🔧 Fix Bugs", variant="primary", size="lg")

                    with gr.Column():
                        fixed_code_output = gr.Textbox(
                            label="✅ Fixed Code & Explanation",
                            lines=10
                        )

                fix_btn.click(
                    fn=fix_bugs_interface,
                    inputs=[code_input],
                    outputs=[fixed_code_output]
                )

            # Code Summarizer Tab
            with gr.TabItem("📝 Code Summarizer"):
                gr.Markdown("""
                ### Get human-readable explanations of complex code.

                **📖 Purpose:** Understand what code does without diving into details
                **🎯 Output:** Clear, non-technical explanations
                """)

                with gr.Row():
                    with gr.Column():
                        code_to_summarize = gr.Textbox(
                            label="💻 Code to Summarize",
                            placeholder="Paste your code here...",
                            lines=10
                        )
                        summarize_btn = gr.Button("📝 Summarize Code", variant="primary", size="lg")

                    with gr.Column():
                        code_summary = gr.Textbox(
                            label="📖 Code Summary & Explanation",
                            lines=10
                        )

                summarize_btn.click(
                    fn=summarize_code_interface,
                    inputs=[code_to_summarize],
                    outputs=[code_summary]
                )

            # Stage Summary Generator Tab
            with gr.TabItem("📊 Stage Summary Generator"):
                gr.Markdown("""
                ### Generate concise summaries for specific SDLC stages.

                **📋 Input:** Requirements for a specific stage
                **📊 Output:** Professional summary with key deliverables
                """)

                with gr.Row():
                    with gr.Column():
                        stage_dropdown = gr.Dropdown(
                            choices=["Planning", "Design", "Implementation", "Testing", "Maintenance"],
                            value="Planning",
                            label="🎯 SDLC Stage"
                        )
                        stage_requirements = gr.Textbox(
                            label="📝 Requirements (one per line)",
                            placeholder="Enter requirements for this stage...",
                            lines=8
                        )
                        stage_summary_btn = gr.Button("📊 Generate Summary", variant="primary", size="lg")

                    with gr.Column():
                        stage_summary_output = gr.Textbox(
                            label="📋 Stage Summary",
                            lines=10
                        )

                stage_summary_btn.click(
                    fn=generate_stage_summary_interface,
                    inputs=[stage_dropdown, stage_requirements],
                    outputs=[stage_summary_output]
                )

        gr.Markdown("""
        ---
        ### 📚 **How to Use This System:**

        1. **📋 Requirements Classification**: Upload a PDF with software requirements
        2. **💻 Code Generation**: Describe what you want to build in natural language
        3. **🐛 Bug Fixing**: Paste problematic code to get fixes and explanations
        4. **📝 Code Summarization**: Get plain English explanations of complex code
        5. **📊 Stage Summaries**: Generate professional summaries for project phases

        **🔧 Technical Details:**
        - **AI Model**: IBM Granite 3.3 2B Instruct
        - **Backend**: FastAPI with async support
        - **Frontend**: Gradio with responsive design
        - **Features**: Enhanced parsing, multi-pass analysis, complete sentence extraction
        """)

    return interface

🚀 Initializing SmartSDLC...
🧠 Loading Granite 3.3 2B Instruct model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

✅ Model loaded successfully on cuda


In [6]:
# Start FastAPI server in background
def start_fastapi():
    print("🌐 Starting FastAPI server...")
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Main function to run the application
def main():
    print("="*60)
    print("🚀 STARTING SMARTSDLC APPLICATION")
    print("="*60)
    print("⚡ AI model initialization complete!")

    # Start FastAPI in a separate thread
    print("🌐 Starting FastAPI server in background...")
    api_thread = threading.Thread(target=start_fastapi, daemon=True)
    api_thread.start()

    print("✅ FastAPI server started on http://localhost:8000")
    print("🎨 Launching Gradio interface...")
    print("="*60)

    # Create and launch Gradio interface
    interface = create_gradio_interface()

    # Launch with public sharing enabled for Colab
    interface.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )

if __name__ == "__main__":
    main()

🚀 STARTING SMARTSDLC APPLICATION
⚡ AI model initialization complete!
🌐 Starting FastAPI server in background...
🌐 Starting FastAPI server...
✅ FastAPI server started on http://localhost:8000
🎨 Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a27aa89efe16259155.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
